### Loading the necessary `libraries`

In [41]:
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()



True

### Creating an `OpenAI client`

In [42]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_chat = os.getenv("OPENAI_MODEL")
model_embedding = os.getenv("MODEL_EMBEDDING")
print(f'API Key loaded: {client.api_key[:5]}***')
print(f'Model loaded: {model_chat}')
print(f'Model embedding loaded: {model_embedding}')

API Key loaded: sk-pr***
Model loaded: gpt-5-nano
Model embedding loaded: text-embedding-ada-002


### Creating a simple `chatbot`

In [ ]:
response = client.chat.completions.create(
    model=model_chat,
    n=3,
   
    messages=[
        {"role": "user", "content": "Write me a python function for adding two number."}
    ]
)


for i, choice in enumerate(response.choices):
    print(f"Response {i + 1}:")
    print(choice.message.content)
    print()

#print(response.choices[0].message.content)


### Creating a Vector `Store`

In [34]:
# Define the vector store name and description
store_name = "Course outlines"
store_description = "Course outlines for a machine learning course"

# Check if a vector store with this name already exists
vector_stores = client.vector_stores.list()
existing_store = None

for vs in vector_stores.data:
    if vs.name == store_name:
        existing_store = vs
        break

if existing_store:
    # Vector store already exists
    print(f"⚠️  Vector store already exists!")
    print(f"Name: {existing_store.name}")
    print(f"ID: {existing_store.id}")
    print(f"Created at: {existing_store.created_at}")
    print(f"File counts: {existing_store.file_counts}")
    vector_store = existing_store
else:
    # Create new vector store
    vector_store = client.vector_stores.create(
        name=store_name,
        description=store_description
    )
    print(f"✅ Vector store created successfully!")
    print(f"Name: {vector_store.name}")
    print(f"ID: {vector_store.id}")

⚠️  Vector store already exists!
Name: Course outlines
ID: vs_696938b687f88191a79aa6b70c012dbb
Created at: 1768503478
File counts: FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


### `Reading` the number of vector stores

In [40]:
vector_stores = client.vector_stores.list()

print(f"Total vector stores: {len(vector_stores.data)}")

for vs in vector_stores.data:
    print(vs.id, vs.name, vs.created_at)

Total vector stores: 1
vs_696938b687f88191a79aa6b70c012dbb Course outlines 1768503478


### Uploading and Attaching file to vector store

In [ ]:
# Use the vector_store.id from the creation cell above
vector_store_id = "vs_696938b687f88191a79aa6b70c012dbb"

# Upload file from course_outlines folder
file = client.files.create(
    file=open(
        "course_outlines/1.docx",  # ✅ Corrected path
        "rb"
    ),
    purpose="assistants"
)

print(f"✅ File uploaded successfully!")
print(f"File ID: {file.id}")
print(f"Filename: {file.filename}")

# Attach file to vector store
vector_file = client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file.id
)

print(f"\n✅ File attached to vector store!")
print(f"Vector Store ID: {vector_store_id}")

# Check ingestion status
files = client.vector_stores.files.list(
    vector_store_id=vector_store_id
)

print("\n📋 Vector store file status:")
for f in files.data:
    print(f"  • File ID: {f.id} | Status: {f.status}")

### Checking the `number` of files attached to vector store

In [37]:
# Get files in the vector store
files = client.vector_stores.files.list(vector_store_id=vector_store.id)

# Count and display
file_count = len(files.data)
print(f"Total files in vector store: {file_count}")

# Show details of each file
if file_count > 0:
    print("\nFile details:")
    for f in files.data:
        print(f"  - File ID: {f.id} | Status: {f.status}")
else:
    print("No files attached to this vector store.")

Total files in vector store: 2

File details:
  - File ID: file-5GBLTfn9rwronU6HCYKkdu | Status: completed
  - File ID: file-L3icTrkHxC5bJRs8m9KK1N | Status: completed


### Searching the Vector `Store`

In [38]:
vector_store_id = "vs_696938b687f88191a79aa6b70c012dbb"
user_query = "What is the document about?"

results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=user_query,
    
)

print("Search results:\n")

for i, result in enumerate(results.data, start=1):
    print(f"Result {i}")
    print(f"Score: {result.score}")
    print(f"Text:\n{result.content[0].text}")
    print("-" * 50)

Search results:

Result 1
Score: 0.795314874323099
Text:
5. Introduction to Artificial Intelligence
· What is AI?
The science of making machines perform tasks that typically require human intelligence.
· History of AI
From early algorithms to modern machine learning and neural networks.
· Applications of AI
In healthcare diagnostics, finance, autonomous vehicles, and virtual assistants.
· Ethical Considerations
Issues around privacy, bias, job displacement, and accountability.
Students will discuss AI tools they interact with daily and analyse potential risks.

6. Troubleshooting Techniques
· Hardware Troubleshooting
Identifying and fixing common issues like faulty RAM, overheating, or connectivity failures.
· Software Troubleshooting
Addressing problems like software crashes, malware infections, and update errors.
Students will perform guided exercises in diagnosing and resolving simulated faults.

7. Hands-on Training in Microsoft Office Suite
· Microsoft Word
Document creation, form

In [39]:
file=open(
        "course_outlines/1.docx",  # ✅ Corrected path
        "rb"
    )

print(file) 

<_io.BufferedReader name='course_outlines/1.docx'>
